# Infrared Meso 1 & 2 + SODAK

In [1]:
##################################################
#
# Libraries
#


from metpy.plots    import colortables
from metpy.plots    import add_timestamp
from datetime       import datetime
from siphon.catalog import TDSCatalog
from datetime       import datetime

import numpy             as np
import os                as os
import pandas            as pd
import pathlib           as pathlib

import metpy
import cartopy.crs       as ccrs
import cartopy.feature   as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import matplotlib.patches as patches




#
##################################################

## Meso Floater 1

In [2]:
##################################################
#
# Control Setup
#

# %load solutions/data_url.py

total_frames = int(45*2 * 0.8)

png_processing_directory = "./temp_files_sat_ir_meso1/"

gif_file_name1 = "./graphics_files/RealTime_SAT_IR_Meso1_Loop.gif"
gif_file_name2 = "./graphics_files/RealTime_SAT_IR_Meso2_Loop.gif"
gif_file_name3 = "./graphics_files/RealTime_SAT_IR_SODAK_Loop.gif"
gif_file_name12 = "./graphics_files/RealTime_SAT_IR_Meso12_Loop.gif"


image_header_label = "GOES 16 Meso1 Band 13 [10.3 µm Clean LW IR Window]"

# Cell content replaced by load magic replacement.

# Create variables for URL generation

image_date = datetime.utcnow().date()
region = 'Mesoscale-1'
channel = 13

# We want to match something like:
# https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

# Construct the data_url string

data_url = ('https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/current/catalog.xml')

# Print out your URL and verify it works!

print(data_url)

#
##################################################

https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/CloudAndMoistureImagery/Mesoscale-1/Channel13/current/catalog.xml


In [3]:
##################################################
#
# Pull Catalog
#

cat = TDSCatalog(data_url)

#
##################################################

In [4]:
##################################################
#
# Create File Inventories
#

file_names_to_retain = list()
file_names_to_use    = list()


for i in range(0,len(cat.datasets[0:total_frames])+1,1) : 
    filename = png_processing_directory + cat.datasets[i].name.replace(".nc",".png")
    file_names_to_retain.append(filename)
    file_names_to_use.append(filename)

        
files_on_hand = [png_processing_directory + s for s in os.listdir(png_processing_directory)]

file_names_to_retain.sort()
file_names_to_use.sort()

file_names_to_use_meso1 = file_names_to_use.copy()


#
##################################################    

In [5]:
##################################################
#
# Clean PNG Directory
#

for filename in files_on_hand:
    if filename not in file_names_to_retain:
        print("Purging ", filename )
        os.remove( filename  )
    else:
        print("Keeping ", filename )
#
##################################################    

Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992019250_e20220992019250_c20220992019250.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992010270_e20220992010270_c20220992010270.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992001250_e20220992001250_c20220992001250.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992020270_e20220992020270_c20220992020270.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992015250_e20220992015250_c20220992015250.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992007250_e20220992007250_c20220992007250.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992013250_e20220992013250_c20220992013250.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992022250_e20220992022250_c20220992022250.png
Keeping  ./temp_files_sat_ir_meso1/OR_ABI-L2-CMIPM1-M6C13_G16_s20220992018250_e202209920

In [6]:
##################################################
#
# Create PNGs
#

for i in range(0,len(cat.datasets[0:total_frames])+1,1) : 

    dataset = cat.datasets[i]
    
    dataset_png_file_name = png_processing_directory + dataset.name.replace(".nc", ".png")
    
    if (not pathlib.Path(dataset_png_file_name).is_file() ):

        ds = dataset.remote_access(use_xarray=True)
        dat = ds.metpy.parse_cf('Sectorized_CMI')
        proj = dat.metpy.cartopy_crs
        x = dat['x']
        y = dat['y']


        tz='America/Denver'
        time_utc   = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
        valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H:%M:%S %Z")
        local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H:%M:%S %Z")

        file_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d_%H%M")


        print(valid_time,local_time)



        fig = plt.figure(figsize=(8, 8), facecolor = 'white')

        plt.suptitle(image_header_label,
                     fontsize = 20, 
                     color    = "black")
        ax = fig.add_subplot(1, 1, 1, projection=proj)
        ax.set_title(valid_time + "  (" + local_time+")",
                        fontsize=15, color="black")
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=2)
        ax.add_feature(cfeature.STATES.with_scale('50m'),    linestyle=':', edgecolor='black')
        ax.add_feature(cfeature.BORDERS.with_scale('50m'),   linewidth=2, edgecolor='black')
        

        # print("range dat = ",np.nanmin(dat.values),np.nanmax(dat.values))
        im = ax.imshow(dat, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper')

        wv_norm, wv_cmap = colortables.get_with_range('WVCIMSS_r', 190, 310)
        im.set_cmap(wv_cmap)
        im.set_norm(wv_norm)

        #########################################
        #
        # Insert a Clock
        #
        
        axins = fig.add_axes(rect     =    [0.065,
                                            0.795,
                                            0.12*0.65306121,
                                            0.12],
                              projection  =  "polar")
        
        time_for_clock = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).time()

        hour   = time_for_clock.hour
        minute = time_for_clock.minute
        second = time_for_clock.second
        
        circle_theta  = np.deg2rad(np.arange(0,360,0.01))
        circle_radius = circle_theta * 0 + 1
        
        if (hour > 12) :
            hour = hour - 12
        
        angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*second/(12*60*60)
        angles_m = 2*np.pi*minute/60+2*np.pi*second/(60*60)
        
        print(time_for_clock)
        print(hour,   np.rad2deg(angles_h))
        print(minute, np.rad2deg(angles_m))

        
        plt.setp(axins.get_yticklabels(), visible=False)
        plt.setp(axins.get_xticklabels(), visible=False)
        axins.spines['polar'].set_visible(False)
        axins.set_ylim(0,1)
        axins.set_theta_zero_location('N')
        axins.set_theta_direction(-1)
        axins.set_facecolor("white")
        axins.grid(False)
        
        axins.plot([angles_h,angles_h], [0,0.6], color="black", linewidth=1.5)
        axins.plot([angles_m,angles_m], [0,0.95], color="black", linewidth=1.5)
        axins.plot(circle_theta, circle_radius, color="darkgrey", linewidth=1)


        
        #
        #########################################             
        plt.tight_layout()
        
        plt.savefig( dataset_png_file_name)
        plt.close()
    else:
        print("We already have this one!")
    

#
##################################################

2022-04-09 20:27:25 UTC 2022-04-09 14:27:25 MDT
14:27:25
2 73.56631455962162
27 164.5


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2592492654.py:101: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 20:26:25 UTC 2022-04-09 14:26:25 MDT
14:26:25
2 73.06631455962163
26 158.5


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2592492654.py:101: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 20:25:25 UTC 2022-04-09 14:25:25 MDT
14:25:25
2 72.56631455962162
25 152.5


...

/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2592492654.py:101: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 19:16:25 UTC 2022-04-09 13:16:25 MDT
13:16:25
1 38.066314559621624
16 98.49999999999999


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2592492654.py:101: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 19:15:25 UTC 2022-04-09 13:15:25 MDT
13:15:25
1 37.566314559621624
15 92.49999999999999


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2592492654.py:101: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [7]:
##################################################
#
# Convert PNGs into an Animated GIF
#


big_string = " ".join(file_names_to_use_meso1)

os.system("convert -delay 10 " + 
          big_string + 
          " " + 
          gif_file_name1)

print("completed "+ gif_file_name1)


#
##################################################

completed ./graphics_files/RealTime_SAT_IR_Meso1_Loop.gif


## Meso Floater 2

In [8]:
##################################################
#
# Control Setup
#

# %load solutions/data_url.py

total_frames = int(45*2 * 0.8)


png_processing_directory = "./temp_files_sat_ir_meso2/"

gif_file_name1 = "./graphics_files/RealTime_SAT_IR_Meso1_Loop.gif"
gif_file_name2 = "./graphics_files/RealTime_SAT_IR_Meso2_Loop.gif"
gif_file_name3 = "./graphics_files/RealTime_SAT_IR_SODAK_Loop.gif"
gif_file_name12 = "./graphics_files/RealTime_SAT_IR_Meso12_Loop.gif"


image_header_label = "GOES 16 Meso2 Band 13 [10.3 µm Clean LW IR Window]"

# Cell content replaced by load magic replacement.

# Create variables for URL generation

image_date = datetime.utcnow().date()
region = 'Mesoscale-2'
channel = 13

# We want to match something like:
# https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

# Construct the data_url string

data_url = ('https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/current/catalog.xml')

# Print out your URL and verify it works!

print(data_url)

#
##################################################

https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/CloudAndMoistureImagery/Mesoscale-2/Channel13/current/catalog.xml


In [9]:
##################################################
#
# Pull Catalog
#

cat = TDSCatalog(data_url)

#
##################################################

In [10]:
##################################################
#
# Create File Inventories
#

file_names_to_retain = list()
file_names_to_use    = list()


for i in range(0,len(cat.datasets[0:total_frames])+1,1) : 
    filename = png_processing_directory + cat.datasets[i].name.replace(".nc",".png")
    file_names_to_retain.append(filename)
    file_names_to_use.append(filename)

        
files_on_hand = [png_processing_directory + s for s in os.listdir(png_processing_directory)]


file_names_to_retain.sort()
file_names_to_use.sort()

file_names_to_use_meso2 = file_names_to_use.copy()


#
##################################################   

In [11]:
##################################################
#
# Clean PNG Directory
#

for filename in files_on_hand:
    if filename not in file_names_to_retain:
        print("Purging ", filename )
        os.remove( filename  )
    else:
        print("Keeping ", filename )
#
##################################################  

Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902317550_e20220902317550_c20220902317550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902303550_e20220902303550_c20220902303550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902311550_e20220902311550_c20220902311550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902305550_e20220902305550_c20220902305550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902321550_e20220902321550_c20220902321550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902309550_e20220902309550_c20220902309550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902316550_e20220902316550_c20220902316550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902302550_e20220902302550_c20220902302550.png
Purging  ./temp_files_sat_ir_meso2/OR_ABI-L2-CMIPM2-M6C02_G16_s20220902310550_e202209023

In [12]:
##################################################
#
# Create PNGs
#

for i in range(0,len(cat.datasets[0:total_frames])+1,1) : 

    dataset = cat.datasets[i]
    
    dataset_png_file_name = png_processing_directory + dataset.name.replace(".nc", ".png")
    
    if (not pathlib.Path(dataset_png_file_name).is_file() ):

        ds = dataset.remote_access(use_xarray=True)
        dat = ds.metpy.parse_cf('Sectorized_CMI')
        proj = dat.metpy.cartopy_crs
        x = dat['x']
        y = dat['y']


        tz='America/Denver'
        time_utc   = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
        valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H:%M:%S %Z")
        local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H:%M:%S %Z")

        file_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d_%H%M")


        print(valid_time,local_time)



        fig = plt.figure(figsize=(8, 8), facecolor = 'white')

        plt.suptitle(image_header_label,
                     fontsize = 20, 
                     color    = "black")
        ax = fig.add_subplot(1, 1, 1, projection=proj)
        ax.set_title(valid_time + "  (" + local_time+")",
                        fontsize=15, color="black")
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=2)
        ax.add_feature(cfeature.STATES.with_scale('50m'),    linestyle=':', edgecolor='black')
        ax.add_feature(cfeature.BORDERS.with_scale('50m'),   linewidth=2, edgecolor='black')
        

        #print("range dat = ",np.nanmin(dat.values),np.nanmax(dat.values))
        im = ax.imshow(dat, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper')

        wv_norm, wv_cmap = colortables.get_with_range('WVCIMSS_r', 190, 310)
        im.set_cmap(wv_cmap)
        im.set_norm(wv_norm)
        
        
        #########################################
        #
        # Insert a Clock
        #
        
        axins = fig.add_axes(rect     =    [0.065,
                                            0.795,
                                            0.12*0.65306121,
                                            0.12],
                              projection  =  "polar")
        
        time_for_clock = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).time()

        hour   = time_for_clock.hour
        minute = time_for_clock.minute
        second = time_for_clock.second
        
        circle_theta  = np.deg2rad(np.arange(0,360,0.01))
        circle_radius = circle_theta * 0 + 1
        
        if (hour > 12) :
            hour = hour - 12
        
        angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*second/(12*60*60)
        angles_m = 2*np.pi*minute/60+2*np.pi*second/(60*60)
        
        print(time_for_clock)
        print(hour,   np.rad2deg(angles_h))
        print(minute, np.rad2deg(angles_m))

        
        plt.setp(axins.get_yticklabels(), visible=False)
        plt.setp(axins.get_xticklabels(), visible=False)
        axins.spines['polar'].set_visible(False)
        axins.set_ylim(0,1)
        axins.set_theta_zero_location('N')
        axins.set_theta_direction(-1)
        axins.set_facecolor("white")
        axins.grid(False)
        
        axins.plot([angles_h,angles_h], [0,0.6], color="black", linewidth=1.5)
        axins.plot([angles_m,angles_m], [0,0.95], color="black", linewidth=1.5)
        axins.plot(circle_theta, circle_radius, color="darkgrey", linewidth=1)


        
        #
        #########################################        
        

        
        plt.tight_layout()
        
        plt.savefig( dataset_png_file_name)
        plt.close()
    else:
        print("We already have this one!")
    

#
##################################################

2022-04-09 20:27:55 UTC 2022-04-09 14:27:55 MDT
14:27:55
2 73.64589203116756
27 167.5


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2778697526.py:105: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 20:26:55 UTC 2022-04-09 14:26:55 MDT
14:26:55
2 73.14589203116758
26 161.5


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2778697526.py:105: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 20:25:55 UTC 2022-04-09 14:25:55 MDT
14:25:55
2 72.64589203116756
25 155.5


...

/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2778697526.py:105: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 19:16:55 UTC 2022-04-09 13:16:55 MDT
13:16:55
1 38.14589203116757
16 101.5


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2778697526.py:105: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 19:15:55 UTC 2022-04-09 13:15:55 MDT
13:15:55
1 37.64589203116757
15 95.49999999999999


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/2778697526.py:105: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [13]:
##################################################
#
# Convert PNGs into an Animated GIF
#


big_string = " ".join(file_names_to_use_meso2)

os.system("convert -delay 15 " + 
          big_string + 
          " " + 
          gif_file_name2)

print("completed "+ gif_file_name2)


#
##################################################

completed ./graphics_files/RealTime_SAT_IR_Meso2_Loop.gif


## South Dakota Images

In [14]:
##################################################
#
# Control Setup
#

# %load solutions/data_url.py

total_frames = int(45*2 * 0.8)


png_processing_directory = "./temp_files_sat_ir_sodak/"

gif_file_name1 = "./graphics_files/RealTime_SAT_IR_Meso1_Loop.gif"
gif_file_name2 = "./graphics_files/RealTime_SAT_IR_Meso2_Loop.gif"
gif_file_name3 = "./graphics_files/RealTime_SAT_IR_SODAK_Loop.gif"
gif_file_name12 = "./graphics_files/RealTime_SAT_IR_Meso12_Loop.gif"


image_header_label = "GOES 16 SODAK Band 13 [10.3 µm Clean LW IR Window]"


# Cell content replaced by load magic replacement.

# Create variables for URL generation

image_date = datetime.utcnow().date()
region = 'CONUS'
channel = 13

# We want to match something like:
# https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

# Construct the data_url string

data_url = ('https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/current/catalog.xml')

# Print out your URL and verify it works!

print(data_url)

#
##################################################

https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel13/current/catalog.xml


In [15]:
##################################################
#
# Pull Catalog
#

cat = TDSCatalog(data_url)

#
##################################################

In [16]:
##################################################
#
# Create File Inventories
#

file_names_to_retain = list()
file_names_to_use    = list()


for i in range(0,len(cat.datasets[0:total_frames])+1,1) : 
    filename = png_processing_directory + cat.datasets[i].name.replace(".nc",".png")
    file_names_to_retain.append(filename)
    file_names_to_use.append(filename)

        
files_on_hand = [png_processing_directory + s for s in os.listdir(png_processing_directory)]


file_names_to_retain.sort()
file_names_to_use.sort()

file_names_to_use_meso3 = file_names_to_use.copy()


#
################################################## 

In [17]:
##################################################
#
# Clean PNG Directory
#

for filename in files_on_hand:
    if filename not in file_names_to_retain:
        print("Purging ", filename )
        os.remove( filename  )
    else:
        print("Keeping ", filename )
#
##################################################  

In [18]:
##################################################
#
# Create PNGs
#

i_rap    = 801
j_rap    = 231
imin_rap = i_rap-250
imax_rap = i_rap+250
jmin_rap = j_rap-250+19
jmax_rap = j_rap+250-19



for i in range(0,len(cat.datasets[0:total_frames])+1,1) : 

    dataset = cat.datasets[i]
    
    dataset_png_file_name = png_processing_directory + dataset.name.replace(".nc", ".png")
    
    if (not pathlib.Path(dataset_png_file_name).is_file() ):

        ds = dataset.remote_access(use_xarray=True)
        dat = ds.metpy.parse_cf('Sectorized_CMI')
        proj = dat.metpy.cartopy_crs
        x = dat['x'][imin_rap:imax_rap]
        y = dat['y'][jmin_rap:jmax_rap]


        tz='America/Denver'
        time_utc   = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
        valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H:%M:%S %Z")
        local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H:%M:%S %Z")

        file_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d_%H%M")


        print(valid_time,local_time)



        fig = plt.figure(figsize=(8, 8), facecolor = 'white')

        plt.suptitle(image_header_label,
                     fontsize = 20, 
                     color    = "black")
        ax = fig.add_subplot(1, 1, 1, projection=proj)
        ax.set_title(valid_time + "  (" + local_time+")",
                        fontsize=15, color="black")
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=2)
        ax.add_feature(cfeature.STATES.with_scale('50m'),    linestyle=':', edgecolor='black')
        ax.add_feature(cfeature.BORDERS.with_scale('50m'),   linewidth=2, edgecolor='black')
        

        #print("range dat = ",np.nanmin(dat.values),np.nanmax(dat.values))
        im = ax.imshow(dat[jmin_rap:jmax_rap,imin_rap:imax_rap], 
                       extent = (x.min(), x.max(), y.min(), y.max()), 
                       origin = 'upper')

        wv_norm, wv_cmap = colortables.get_with_range('WVCIMSS_r', 190, 310)
        im.set_cmap(wv_cmap)
        im.set_norm(wv_norm)
        
        #########################################
        #
        # Insert a Clock
        #
        
        axins = fig.add_axes(rect     =    [0.02,
                                            0.81,
                                            0.12*0.65306121,
                                            0.12],
                              projection  =  "polar")
        
        time_for_clock = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).time()

        hour   = time_for_clock.hour
        minute = time_for_clock.minute
        second = time_for_clock.second
        
        circle_theta  = np.deg2rad(np.arange(0,360,0.01))
        circle_radius = circle_theta * 0 + 1
        
        if (hour > 12) :
            hour = hour - 12
        
        angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*second/(12*60*60)
        angles_m = 2*np.pi*minute/60+2*np.pi*second/(60*60)
        
        print(time_for_clock)
        print(hour,   np.rad2deg(angles_h))
        print(minute, np.rad2deg(angles_m))

        
        plt.setp(axins.get_yticklabels(), visible=False)
        plt.setp(axins.get_xticklabels(), visible=False)
        axins.spines['polar'].set_visible(False)
        axins.set_ylim(0,1)
        axins.set_theta_zero_location('N')
        axins.set_theta_direction(-1)
        axins.set_facecolor("white")
        axins.grid(False)
        
        axins.plot([angles_h,angles_h], [0,0.6], color="black", linewidth=1.5)
        axins.plot([angles_m,angles_m], [0,0.95], color="black", linewidth=1.5)
        axins.plot(circle_theta, circle_radius, color="darkgrey", linewidth=1)


        
        #
        #########################################             

        
        plt.tight_layout()
        
        plt.savefig( dataset_png_file_name)
        plt.close()
    else:
        print("We already have this one!")
    

#
##################################################

2022-04-09 20:26:17 UTC 2022-04-09 14:26:17 MDT
14:26:17
2 73.04509390054271
26 157.70000000000002


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/1367384786.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 20:21:17 UTC 2022-04-09 14:21:17 MDT
14:21:17
2 70.5450939005427
21 127.69999999999999


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/1367384786.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 20:16:17 UTC 2022-04-09 14:16:17 MDT
14:16:17
2 68.0450939005427
16 97.69999999999999


...

/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/1367384786.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 14:31:17 UTC 2022-04-09 08:31:17 MDT
08:31:17
8 255.5450939005427
31 187.7


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/1367384786.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


2022-04-09 14:26:17 UTC 2022-04-09 08:26:17 MDT
08:26:17
8 253.0450939005427
26 157.70000000000002


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_4373/1367384786.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [19]:
##################################################
#
# Convert PNGs into an Animated GIF
#


big_string = " ".join(file_names_to_use_meso3)

os.system("convert -delay 15 " + 
          big_string + 
          " " + 
          gif_file_name3)

print("completed "+ gif_file_name3)


#
##################################################

completed ./graphics_files/RealTime_SAT_IR_SODAK_Loop.gif


In [20]:
##################################################
#
# Convert PNGs into an Animated GIF
#

file_names_to_use_meso12 = file_names_to_use_meso3 + file_names_to_use_meso1 + file_names_to_use_meso2

big_string = " ".join(file_names_to_use_meso12)

os.system("convert -delay 10 " + 
          big_string + 
          " " + 
          gif_file_name12)

print("completed "+gif_file_name12)

#
##################################################

completed ./graphics_files/RealTime_SAT_IR_Meso12_Loop.gif


In [21]:
print("--- GOES SD WKT" )
print(proj.to_wkt())
print("--- GOES SD Extent")
print(ax.get_aspect())

--- GOES SD WKT
PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",ELLIPSOID["GRS 1980",6378137,298.257222096042,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["unknown",METHOD["Geostationary Satellite (Sweep X)"],PARAMETER["Longitude of natural origin",-75,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Satellite Height",35786023,LENGTHUNIT["metre",1,ID["EPSG",9001]]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]],REMARK["PROJ CRS string: +proj=geos +a=6378137.0 +b=6356752.31414 +lon_0=-75.00000000000001 +lat_0=0.0 +h=35786023.0 +x_0=0 +y_0=0 +units=m +sweep=x +no_defs"]]
--- GOES SD Extent
1.0
